In [6]:
import cv2

# Load the image
image = cv2.imread(r'C:\Users\25868\Desktop\smoke\Screenshot_20240522-120940.png')

# Get the dimensions of the image
image_height, image_width = image.shape[:2]

# Calculate the scaling factor to fit the image within a reasonable window size
scaling_factor = min(1.0, 600.0 / max(image_height, image_width))

# Resize the image
resized_image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor)

# Create a window and display the resized image
cv2.imshow('Resized Image', resized_image)

# Initialize list to store path coordinates
path_points = []

# Mouse callback function to capture mouse events
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Add clicked point to the path
        path_points.append((x, y))
        # Draw a circle at the clicked point
        cv2.circle(resized_image, (x, y), 5, (0, 0, 255), -1)
        # Display the updated image
        cv2.imshow('Resized Image', resized_image)

# Set mouse callback function
cv2.setMouseCallback('Resized Image', mouse_callback)

# Wait for the user to annotate the path
cv2.waitKey(0)

# Print the coordinates of the path
print("Path coordinates:", path_points)

# Close all windows
cv2.destroyAllWindows()


Path coordinates: [(192, 39), (195, 94), (194, 164), (208, 247), (225, 360), (269, 479), (316, 543), (351, 571)]


In [6]:
import cv2
import numpy as np

# Define the arbitrary path (list of points)
path_points = [(205, 35), (205, 80), (205, 144), (205, 238), (205, 304), (210, 370), (225, 430), (240, 486), (261, 528), (294, 565), (320, 592)]

# Define parameters for motion analysis
max_deviation = 20  # Maximum deviation allowed from the path
max_stay_time = 10  # Maximum consecutive frames allowed to stay in one place

# Function to calculate distance between two points
def distance(point1, point2):
    return np.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

# Function to check if a point is near the path
def point_near_path(point, path_points, max_deviation):
    # Check distance from point to each line segment of the path
    for i in range(len(path_points) - 1):
        start = path_points[i]
        end = path_points[i + 1]
        dist = abs((end[1] - start[1]) * point[0] - (end[0] - start[0]) * point[1] + end[0] * start[1] - end[1] * start[0]) / distance(start, end)
        if dist < max_deviation:
            return True
    return False

# Load video file
cap = cv2.VideoCapture(r'./zoom.mp4')

# Initialize variables for object tracking and alert
prev_centroid = None
prev_frame = None
stay_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Perform object tracking (centroid tracking)
    if prev_frame is not None:
        # Calculate absolute difference between current and previous frame
        diff = cv2.absdiff(gray, prev_frame)
        
        # Threshold the difference to obtain binary image
        _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
        
        # Find contours in the binary image
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Check if any contours are found
        if contours:
            # Assume only one contour (smoke) for simplicity
            contour = contours[0]
            
            # Calculate centroid of the contour
            M = cv2.moments(contour)
            if M['m00'] != 0:
                centroid = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
                
                # Check if centroid is near the path
                if point_near_path(centroid, path_points, max_deviation):
                    # Check if smoke stayed in one place for too long
                    if prev_centroid is not None and distance(prev_centroid, centroid) < 5:
                        stay_count += 1
                        if stay_count > max_stay_time:
                            print("Smoke stayed in one place for too long!")
                    else:
                        stay_count = 0
                    
                    # Update previous centroid
                    prev_centroid = centroid
                
                # Draw centroid on frame
                cv2.circle(frame, centroid, 5, (0, 255, 0), -1)
    
    # Draw path on frame
    for i in range(len(path_points) - 1):
        cv2.line(frame, path_points[i], path_points[i + 1], (0, 0, 255), 2)
    
    # Show frame
    cv2.imshow('Frame', frame)
    
    # Update previous frame
    prev_frame = gray.copy()
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2

# Load the image
image = cv2.imread(r'C:\Users\25868\Desktop\smoke\Screenshot_20240522-120940.png')

# Get the dimensions of the image
image_height, image_width = image.shape[:2]

# Calculate the scaling factor to fit the image within a reasonable window size
scaling_factor = min(1.0, 600.0 / max(image_height, image_width))

# Resize the image
resized_image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor)

# Create a window and display the resized image
cv2.imshow('Resized Image', resized_image)

# Initialize list to store path coordinates
path_points = []

# Mouse callback function to capture mouse events
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Add clicked point to the path
        path_points.append((x, y))
        # Draw a circle at the clicked point
        cv2.circle(resized_image, (x, y), 5, (0, 0, 255), -1)
        # Display the updated image
        cv2.imshow('Resized Image', resized_image)

# Set mouse callback function
cv2.setMouseCallback('Resized Image', mouse_callback)

# Wait for the user to annotate the path
cv2.waitKey(0)

# Print the coordinates of the path
print("Path coordinates:", path_points)

# Close all windows
cv2.destroyAllWindows()


Path coordinates: [(195, 38), (195, 83), (195, 135), (195, 193), (195, 252), (196, 304), (200, 361), (210, 434), (225, 489), (250, 539), (293, 581), (319, 596)]


In [5]:
import cv2
import subprocess
import numpy as np

# Input video path
input_video_path = "./resize.mp4"

# Get video properties using FFprobe
ffprobe_cmd = [
    'ffprobe', '-v', 'error', '-select_streams', 'v:0',
    '-show_entries', 'stream=width,height', '-of', 'csv=p=0', input_video_path
]
video_properties = subprocess.check_output(ffprobe_cmd).decode().strip().split(',')
video_width, video_height = map(int, video_properties)

# FFmpeg command to decode video frames
ffmpeg_cmd = [
    'ffmpeg', '-i', input_video_path, '-f', 'image2pipe', '-pix_fmt', 'bgr24', '-vcodec', 'rawvideo', '-'
]

# Open FFmpeg process
process = subprocess.Popen(ffmpeg_cmd, stdout=subprocess.PIPE)

# Define output video writer
output_video_path = "main.mp4"
output_video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (video_width, video_height))

# Read and write each frame of the video
while True:
    # Read raw video data from FFmpeg process
    raw_frame = process.stdout.read(video_width * video_height * 3)
    if not raw_frame:
        break

    # Convert raw video data to numpy array
    frame = np.frombuffer(raw_frame, dtype='uint8').reshape((video_height, video_width, 3))

    # Write the frame to the output video file
    output_video_writer.write(frame)

    # Display the frame (optional)
    cv2.imshow('Decoded Video', frame)

    # Wait for a key press and check if the user pressed 'q' to exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the output video writer
output_video_writer.release()

# Close OpenCV window
cv2.destroyAllWindows()

print(f"Video saved as {output_video_path}")


Video saved as main.mp4


In [9]:
import cv2
import numpy as np

# Load the image
image = cv2.imread(r'C:\Users\25868\Desktop\smoke\Screenshot_20240522-120940.png')

# Get the dimensions of the image
image_height, image_width = image.shape[:2]

# Calculate the scaling factor to fit the image within a reasonable window size
scaling_factor = min(1.0, 600.0 / max(image_height, image_width))

# Resize the image
resized_image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor)

# Create a window and display the resized image
cv2.imshow('Resized Image', resized_image)

# Initialize variables for the path
path_lines = []
first_line_defined = False

# Mouse callback function to capture mouse events
def mouse_callback(event, x, y, flags, param):
    global first_line_defined, path_lines

    if event == cv2.EVENT_LBUTTONDOWN:
        if not first_line_defined:
            if len(path_lines) == 0:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the first line:", (x, y))
        else:
            if len(path_lines) == 1:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the second line:", (x, y))

# Set mouse callback function
cv2.setMouseCallback('Resized Image', mouse_callback)

# Main loop to define the path lines
while True:
    # Read keyboard input
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == 13:  # Enter key
        if not first_line_defined:
            if len(path_lines[-1]) >= 2:
                first_line_defined = True
                print("First line defined:", path_lines[-1])
            else:
                print("Please define at least 2 points for the first line.")
        else:
            if len(path_lines[-1]) >= 2:
                print("Second line defined:", path_lines[-1])
                break
            else:
                print("Please define at least 2 points for the second line.")

    # Draw the defined path lines and points
    if path_lines:
        resized_image_with_lines = resized_image.copy()
        for line in path_lines:
            if len(line) > 1:
                for i in range(len(line) - 1):
                    cv2.line(resized_image_with_lines, line[i], line[i + 1], (0, 255, 0), 2)
                    cv2.circle(resized_image_with_lines, line[i], 5, (255, 0, 0), -1)  # Draw circle at each point
                cv2.circle(resized_image_with_lines, line[-1], 5, (255, 0, 0), -1)  # Draw circle at last point
        cv2.imshow('Resized Image', resized_image_with_lines)

# Close all windows
cv2.destroyAllWindows()


Point added to the first line: (153, 33)
Point added to the first line: (152, 76)
Point added to the first line: (152, 119)
Point added to the first line: (152, 151)
Point added to the first line: (150, 202)
Point added to the first line: (149, 248)
Point added to the first line: (154, 295)
Point added to the first line: (163, 345)
Point added to the first line: (170, 406)
Point added to the first line: (177, 439)
Point added to the first line: (205, 498)
Point added to the first line: (230, 536)
Point added to the first line: (249, 574)
Point added to the first line: (268, 598)
First line defined: [(153, 33), (152, 76), (152, 119), (152, 151), (150, 202), (149, 248), (154, 295), (163, 345), (170, 406), (177, 439), (205, 498), (230, 536), (249, 574), (268, 598)]
Point added to the second line: (231, 35)
Point added to the second line: (232, 71)
Point added to the second line: (234, 117)
Point added to the second line: (236, 160)
Point added to the second line: (237, 211)
Point added to

In [32]:
import cv2
import numpy as np
import subprocess

# Function to increase contrast of frame
def increase_contrast(frame, clip_limit=2.0, tile_grid_size=(8, 8)):
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    contrast_frame = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return contrast_frame

# Function to process video and detect smoke
def process_video(input_path, output_path, blur_ksize=(21, 21), blur_sigma=0, threshold_value=30):
    # Open the video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the video frame width, height, and frames per second (fps)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Prepare FFmpeg command
    ffmpeg_command = [
        'ffmpeg',
        '-y',  # Overwrite output file if exists
        '-f', 'rawvideo',
        '-vcodec', 'rawvideo',
        '-pix_fmt', 'bgr24',  # Changed to BGR24 to include color drawing
        '-s', f"{frame_width}x{frame_height}",  # Frame size
        '-r', str(int(fps)),  # Frame rate
        '-i', '-',  # Input from pipe
        '-an',  # No audio
        '-vcodec', 'libx264',
        '-pix_fmt', 'yuv420p',
        output_path
    ]

    # Start FFmpeg process
    process = subprocess.Popen(ffmpeg_command, stdin=subprocess.PIPE)

    prev_frame = None
    deviation_timer = 0
    deviation_threshold = 5  # seconds

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Increase contrast of the frame
        contrast_frame = increase_contrast(frame)

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(contrast_frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale frame
        blurred_frame = cv2.GaussianBlur(gray_frame, blur_ksize, blur_sigma)

        if prev_frame is None:
            # For the first frame, there's no previous frame to compare with
            delta_frame = np.zeros_like(blurred_frame)
            thresholded_frame = np.zeros_like(blurred_frame)
        else:
            # Calculate the absolute difference between the current frame and the previous frame
            delta_frame = cv2.absdiff(blurred_frame, prev_frame)

            # Apply threshold to the delta frame
            _, thresholded_frame = cv2.threshold(delta_frame, threshold_value, 255, cv2.THRESH_BINARY)

            # Get the coordinates of the white pixels in the thresholded frame
            white_pixels = np.column_stack(np.where(thresholded_frame == 255))

            # Draw green points at the coordinates of the white pixels
            for pixel in white_pixels:
                frame[pixel[0], pixel[1]] = [0, 255, 0]  # Draw green point

            # Check if smoke flow is within path
            if check_smoke_flow_within_path(white_pixels, path_lines):
                deviation_timer = 0  # Reset deviation timer if smoke flow is within path
            else:
                deviation_timer += 1  # Increment deviation timer

            # Check for deviation
            if deviation_timer >= deviation_threshold * fps:  # Convert threshold to frames
                print("Smoke deviation detected!")
                deviation_timer = 0  # Reset timer after detection

        # Write the frame to FFmpeg process
        process.stdin.write(frame.tobytes())

        # Update previous frame
        prev_frame = blurred_frame

    # Close FFmpeg process
    process.stdin.close()
    process.wait()

    # Release the video capture
    cap.release()
    cv2.destroyAllWindows()

# Function to check if smoke flow is within path
def check_smoke_flow_within_path(smoke_coordinates, path_lines):
    contour_points = np.vstack(path_lines).astype(np.float32)  # Vertically stack all points
    contour = contour_points.reshape((-1, 1, 2))  # Reshape to contour format
    print("Contour shape:", contour.shape)
    print("Contour type:", contour.dtype)
    for coord in smoke_coordinates:
        coord_tuple = (float(coord[1]), float(coord[0]))  # Convert to tuple of floats (x, y)
        print("Coord:", coord_tuple)
        if cv2.pointPolygonTest(contour, coord_tuple, False) < 0:
            return False
    return True

# Load the image
image = cv2.imread(r'C:\Users\25868\Desktop\smoke\Screenshot_20240522-120940.png')

# Get the dimensions of the image
image_height, image_width = image.shape[:2]

# Calculate the scaling factor to fit the image within a reasonable window size
scaling_factor = min(1.0, 600.0 / max(image_height, image_width))

# Resize the image
resized_image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor)

# Create a window and display the resized image
cv2.imshow('Resized Image', resized_image)

# Initialize variables for the path
path_lines = []
first_line_defined = False

# Mouse callback function to capture mouse events
def mouse_callback(event, x, y, flags, param):
    global first_line_defined, path_lines

    if event == cv2.EVENT_LBUTTONDOWN:
        if not first_line_defined:
            if len(path_lines) == 0:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the first line:", (x, y))
        else:
            if len(path_lines) == 1:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the second line:", (x, y))

# Set mouse callback function
cv2.setMouseCallback('Resized Image', mouse_callback)

# Main loop to define the path lines
while True:
    # Read keyboard input
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == 13:  # Enter key
        if not first_line_defined:
            if len(path_lines[-1]) >= 2:
                first_line_defined = True
                print("First line defined:", path_lines[-1])
            else:
                print("Please define at least 2 points for the first line.")
        else:
            if len(path_lines[-1]) >= 2:
                print("Second line defined:", path_lines[-1])
                break
            else:
                print("Please define at least 2 points for the second line.")

    # Draw the defined path lines and points
    if path_lines:
        resized_image_with_lines = resized_image.copy()
        for line in path_lines:
            if len(line) > 1:
                for i in range(len(line) - 1):
                    cv2.line(resized_image_with_lines, line[i], line[i + 1], (0, 255, 0), 2)
                    cv2.circle(resized_image_with_lines, line[i], 5, (255, 0, 0), -1)  # Draw circle at each point
                cv2.circle(resized_image_with_lines, line[-1], 5, (255, 0, 0), -1)  # Draw circle at last point
        cv2.imshow('Resized Image', resized_image_with_lines)

# Close all windows
cv2.destroyAllWindows()

# Path to input video file
input_video_path = './zoom.mp4'

# Path to output video file
output_video_path = './7.mp4'

# Call the function to process the video with a threshold value of 30
process_video(input_video_path, output_video_path, threshold_value=10)


Point added to the first line: (134, 33)
Point added to the first line: (139, 225)
Point added to the first line: (154, 383)
Point added to the first line: (215, 507)
Point added to the first line: (269, 595)
First line defined: [(134, 33), (139, 225), (154, 383), (215, 507), (269, 595)]
Point added to the second line: (225, 33)
Point added to the second line: (235, 212)
Point added to the second line: (241, 342)
Point added to the second line: (263, 413)
Point added to the second line: (304, 493)
Point added to the second line: (351, 573)
Point added to the second line: (368, 598)
Second line defined: [(225, 33), (235, 212), (241, 342), (263, 413), (304, 493), (351, 573), (368, 598)]
Contour shape: (12, 1, 2)
Contour type: float32
Coord: (533.0, 0.0)
Contour shape: (12, 1, 2)
Contour type: float32
Coord: (99.0, 245.0)
Contour shape: (12, 1, 2)
Contour type: float32
Coord: (606.0, 216.0)
Contour shape: (12, 1, 2)
Contour type: float32
Coord: (287.0, 542.0)
Contour shape: (12, 1, 2)
Con

In [56]:
import cv2
import numpy as np
import subprocess

# Function to increase contrast of frame
def increase_contrast(frame, clip_limit=2.0, tile_grid_size=(8, 8)):
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    contrast_frame = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return contrast_frame

# Function to process video and detect smoke
def process_video(input_path, output_path, blur_ksize=(21, 21), blur_sigma=0, threshold_value=30):
    # Open the video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the video frame width, height, and frames per second (fps)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Prepare FFmpeg command
    ffmpeg_command = [
        'ffmpeg',
        '-y',  # Overwrite output file if exists
        '-f', 'rawvideo',
        '-vcodec', 'rawvideo',
        '-pix_fmt', 'bgr24',  # Changed to BGR24 to include color drawing
        '-s', f"{frame_width}x{frame_height}",  # Frame size
        '-r', str(int(fps)),  # Frame rate
        '-i', '-',  # Input from pipe
        '-an',  # No audio
        '-vcodec', 'libx264',
        '-pix_fmt', 'yuv420p',
        output_path
    ]

    # Start FFmpeg process
    process = subprocess.Popen(ffmpeg_command, stdin=subprocess.PIPE)

    prev_frame = None
    deviation_timer = 0
    deviation_threshold = 5  # seconds

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Increase contrast of the frame
        contrast_frame = increase_contrast(frame)

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(contrast_frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale frame
        blurred_frame = cv2.GaussianBlur(gray_frame, blur_ksize, blur_sigma)

        if prev_frame is None:
            # For the first frame, there's no previous frame to compare with
            delta_frame = np.zeros_like(blurred_frame)
            thresholded_frame = np.zeros_like(blurred_frame)
        else:
            # Calculate the absolute difference between the current frame and the previous frame
            delta_frame = cv2.absdiff(blurred_frame, prev_frame)

            # Apply threshold to the delta frame
            _, thresholded_frame = cv2.threshold(delta_frame, threshold_value, 255, cv2.THRESH_BINARY)

            # Get the coordinates of the white pixels in the thresholded frame
            white_pixels = np.column_stack(np.where(thresholded_frame == 255))

            # Draw green points at the coordinates of the white pixels
            for pixel in white_pixels:
                frame[pixel[0], pixel[1]] = [0, 255, 0]  # Draw green point

            # Check if smoke flow is within path
            if check_smoke_flow_within_path(white_pixels, path_lines):
                deviation_timer = 0  # Reset deviation timer if smoke flow is within path
            else:
                deviation_timer += 1  # Increment deviation timer

            # Check for deviation
            if deviation_timer >= deviation_threshold * fps:  # Convert threshold to frames
                print("Smoke deviation detected!")
                deviation_timer = 0  # Reset timer after detection

        # Draw the defined path lines on the frame
        draw_path_lines(frame, path_lines)

        # Write the frame to FFmpeg process
        process.stdin.write(frame.tobytes())

        # Update previous frame
        prev_frame = blurred_frame

    # Close FFmpeg process
    process.stdin.close()
    process.wait()

    # Release the video capture
    cap.release()
    cv2.destroyAllWindows()

# Function to check if smoke flow is within path
def check_smoke_flow_within_path(smoke_coordinates, path_lines):
    contour_points = np.vstack(path_lines).astype(np.float32)  # Vertically stack all points
    contour = contour_points.reshape((-1, 1, 2))  # Reshape to contour format
    for coord in smoke_coordinates:
        coord_tuple = (float(coord[1]), float(coord[0]))  # Convert to tuple of floats (x, y)
        if cv2.pointPolygonTest(contour, coord_tuple, False) < 0:
            return False
    return True

# Function to draw path lines on a frame
def draw_path_lines(frame, path_lines):
    for line in path_lines:
        if len(line) > 1:
            for i in range(len(line) - 1):
                cv2.line(frame, line[i], line[i + 1], (0, 255, 0), 2)
                cv2.circle(frame, line[i], 5, (255, 0, 0), -1)  # Draw circle at each point
            cv2.circle(frame, line[-1], 5, (255, 0, 0), -1)  # Draw circle at last point

# Load the image
image = cv2.imread(r'C:\Users\25868\Desktop\smoke\Screenshot_20240522-120940.png')

# Get the dimensions of the image
image_height, image_width = image.shape[:2]

# Calculate the scaling factor to fit the image within a reasonable window size
scaling_factor = min(1.0, 600.0 / max(image_height, image_width))

# Resize the image
resized_image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor)

# Create a window and display the resized image
cv2.imshow('Resized Image', resized_image)

# Initialize variables for the path
path_lines = []
first_line_defined = False

# Mouse callback function to capture mouse events
def mouse_callback(event, x, y, flags, param):
    global first_line_defined, path_lines

    if event == cv2.EVENT_LBUTTONDOWN:
        if not first_line_defined:
            if len(path_lines) == 0:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the first line:", (x, y))
        else:
            if len(path_lines) == 1:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the second line:", (x, y))

# Set mouse callback function
cv2.setMouseCallback('Resized Image', mouse_callback)

# Main loop to define the path lines
while True:
    # Read keyboard input
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == 13:  # Enter key
        if not first_line_defined:
            if len(path_lines[-1]) >= 2:
                first_line_defined = True
                print("First line defined:", path_lines[-1])
            else:
                print("Please define at least 2 points for the first line.")
        else:
            if len(path_lines[-1]) >= 2:
                print("Second line defined:", path_lines[-1])
                break
            else:
                print("Please define at least 2 points for the second line.")

    # Draw the defined path lines and points
    if path_lines:
        resized_image_with_lines = resized_image.copy()
        for line in path_lines:
            if len(line) > 1:
                for i in range(len(line) - 1):
                    cv2.line(resized_image_with_lines, line[i], line[i + 1], (0, 255, 0), 2)
                    cv2.circle(resized_image_with_lines, line[i], 5, (255, 0, 0), -1)  # Draw circle at each point
                cv2.circle(resized_image_with_lines, line[-1], 5, (255, 0, 0), -1)  # Draw circle at last point
        cv2.imshow('Resized Image', resized_image_with_lines)

# Close all windows
cv2.destroyAllWindows()

# Path to input video file
input_video_path = './zoom.mp4'

# Path to output video file
output_video_path = './7.mp4'

# Call the function to process the video with a threshold value of 30
process_video(input_video_path, output_video_path, threshold_value=10)


Point added to the first line: (147, 32)
Point added to the first line: (147, 153)
Point added to the first line: (154, 267)
Point added to the first line: (174, 383)
Point added to the first line: (196, 468)
Point added to the first line: (222, 550)
Point added to the first line: (243, 598)
First line defined: [(147, 32), (147, 153), (154, 267), (174, 383), (196, 468), (222, 550), (243, 598)]
Point added to the second line: (266, 32)
Point added to the second line: (266, 176)
Point added to the second line: (273, 294)
Point added to the second line: (284, 354)
Point added to the second line: (308, 455)
Point added to the second line: (334, 525)
Point added to the second line: (395, 596)
Second line defined: [(266, 32), (266, 176), (273, 294), (284, 354), (308, 455), (334, 525), (395, 596)]
Smoke deviation detected!
Smoke deviation detected!


In [38]:
import cv2
import numpy as np
import subprocess

# Function to increase contrast of frame
def increase_contrast(frame, clip_limit=2.0, tile_grid_size=(8, 8)):
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    contrast_frame = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return contrast_frame

# Function to process video and detect smoke
def process_video(input_path, output_path, blur_ksize=(21, 21), blur_sigma=0, threshold_value=30):
    # Open the video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the video frame width, height, and frames per second (fps)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Prepare FFmpeg command
    ffmpeg_command = [
        'ffmpeg',
        '-y',  # Overwrite output file if exists
        '-f', 'rawvideo',
        '-vcodec', 'rawvideo',
        '-pix_fmt', 'bgr24',  # Changed to BGR24 to include color drawing
        '-s', f"{frame_width}x{frame_height}",  # Frame size
        '-r', str(int(fps)),  # Frame rate
        '-i', '-',  # Input from pipe
        '-an',  # No audio
        '-vcodec', 'libx264',
        '-pix_fmt', 'yuv420p',
        output_path
    ]

    # Start FFmpeg process
    process = subprocess.Popen(ffmpeg_command, stdin=subprocess.PIPE)

    prev_frame = None
    deviation_timer = 0
    deviation_threshold = 5  # seconds

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Increase contrast of the frame
        contrast_frame = increase_contrast(frame)

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(contrast_frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale frame
        blurred_frame = cv2.GaussianBlur(gray_frame, blur_ksize, blur_sigma)

        if prev_frame is None:
            # For the first frame, there's no previous frame to compare with
            delta_frame = np.zeros_like(blurred_frame)
            thresholded_frame = np.zeros_like(blurred_frame)
        else:
            # Calculate the absolute difference between the current frame and the previous frame
            delta_frame = cv2.absdiff(blurred_frame, prev_frame)

            # Apply threshold to the delta frame
            _, thresholded_frame = cv2.threshold(delta_frame, threshold_value, 255, cv2.THRESH_BINARY)

            # Get the coordinates of the white pixels in the thresholded frame
            white_pixels = np.column_stack(np.where(thresholded_frame == 255))

            # Draw green points at the coordinates of the white pixels
            for pixel in white_pixels:
                frame[pixel[0], pixel[1]] = [0, 255, 0]  # Draw green point

            # Check if smoke flow is within path
            if check_smoke_flow_within_path(white_pixels, path_lines):
                deviation_timer = 0  # Reset deviation timer if smoke flow is within path
                print("Smoke within range")
            else:
                deviation_timer += 1  # Increment deviation timer

            # Check for deviation
            if deviation_timer >= deviation_threshold * fps:  # Convert threshold to frames
                print("Smoke deviation detected!")
                deviation_timer = 0  # Reset timer after detection

        # Draw the defined path lines on the frame
        draw_path_lines(frame, path_lines)

        # Write the frame to FFmpeg process
        process.stdin.write(frame.tobytes())

        # Update previous frame
        prev_frame = blurred_frame

    # Close FFmpeg process
    process.stdin.close()
    process.wait()

    # Release the video capture
    cap.release()
    cv2.destroyAllWindows()

# Function to check if smoke flow is within path
def check_smoke_flow_within_path(smoke_coordinates, path_lines):
    contour_points = np.vstack(path_lines).astype(np.float32)  # Vertically stack all points
    contour = contour_points.reshape((-1, 1, 2))  # Reshape to contour format
    for coord in smoke_coordinates:
        coord_tuple = (float(coord[1]), float(coord[0]))  # Convert to tuple of floats (x, y)
        if cv2.pointPolygonTest(contour, coord_tuple, False) < 0:
            return False
    return True

# Function to draw path lines on a frame
def draw_path_lines(frame, path_lines):
    for line in path_lines:
        if len(line) > 1:
            for i in range(len(line) - 1):
                cv2.line(frame, line[i], line[i + 1], (0, 255, 0), 2)
                cv2.circle(frame, line[i], 5, (255, 0, 0), -1)  # Draw circle at each point
            cv2.circle(frame, line[-1], 5, (255, 0, 0), -1)  # Draw circle at last point

# Load the image
image = cv2.imread(r'C:\Users\25868\Desktop\smoke\Screenshot_20240522-120940.png')

# Get the dimensions of the image
image_height, image_width = image.shape[:2]

# Calculate the scaling factor to fit the image within a reasonable window size
scaling_factor = min(1.0, 600.0 / max(image_height, image_width))

# Resize the image
resized_image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor)

# Create a window and display the resized image
cv2.imshow('Resized Image', resized_image)

# Initialize variables for the path
path_lines = []
first_line_defined = False

# Mouse callback function to capture mouse events
def mouse_callback(event, x, y, flags, param):
    global first_line_defined, path_lines

    if event == cv2.EVENT_LBUTTONDOWN:
        if not first_line_defined:
            if len(path_lines) == 0:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the first line:", (x, y))
        else:
            if len(path_lines) == 1:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the second line:", (x, y))

# Set mouse callback function
cv2.setMouseCallback('Resized Image', mouse_callback)

# Main loop to define the path lines
while True:
    # Read keyboard input
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == 13:  # Enter key
        if not first_line_defined:
            if len(path_lines[-1]) >= 2:
                first_line_defined = True
                print("First line defined:", path_lines[-1])
            else:
                print("Please define at least 2 points for the first line.")
        else:
            if len(path_lines[-1]) >= 2:
                print("Second line defined:", path_lines[-1])
                break
            else:
                print("Please define at least 2 points for the second line.")

    # Draw the defined path lines and points
    if path_lines:
        resized_image_with_lines = resized_image.copy()
        for line in path_lines:
            if len(line) > 1:
                for i in range(len(line) - 1):
                    cv2.line(resized_image_with_lines, line[i], line[i + 1], (0, 255, 0), 2)
                    cv2.circle(resized_image_with_lines, line[i], 5, (255, 0, 0), -1)  # Draw circle at each point
                cv2.circle(resized_image_with_lines, line[-1], 5, (255, 0, 0), -1)  # Draw circle at last point
        cv2.imshow('Resized Image', resized_image_with_lines)

# Close all windows
cv2.destroyAllWindows()

# Path to input video file
input_video_path = './zoom.mp4'

# Path to output video file
output_video_path = './8.mp4'

# Call the function to process the video with a threshold value of 30
process_video(input_video_path, output_video_path, threshold_value=10)


Point added to the first line: (132, 30)
Point added to the first line: (131, 185)
Point added to the first line: (132, 296)
Point added to the first line: (143, 387)
Point added to the first line: (168, 529)
Point added to the first line: (189, 599)
First line defined: [(132, 30), (131, 185), (132, 296), (143, 387), (168, 529), (189, 599)]
Point added to the second line: (419, 34)
Point added to the second line: (419, 117)
Point added to the second line: (424, 198)
Point added to the second line: (423, 300)
Point added to the second line: (426, 409)
Point added to the second line: (427, 504)
Point added to the second line: (428, 594)
Second line defined: [(419, 34), (419, 117), (424, 198), (423, 300), (426, 409), (427, 504), (428, 594)]
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke

In [57]:
import cv2
import numpy as np
import subprocess

# Function to increase contrast of frame
def increase_contrast(frame, clip_limit=2.0, tile_grid_size=(8, 8)):
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    contrast_frame = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return contrast_frame

# Function to process video and detect smoke
def process_video(input_path, output_path, path_lines, blur_ksize=(21, 21), blur_sigma=0, threshold_value=30):
    # Open the video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the video frame width, height, and frames per second (fps)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Prepare FFmpeg command
    ffmpeg_command = [
        'ffmpeg',
        '-y',  # Overwrite output file if exists
        '-f', 'rawvideo',
        '-vcodec', 'rawvideo',
        '-pix_fmt', 'bgr24',  # Changed to BGR24 to include color drawing
        '-s', f"{frame_width}x{frame_height}",  # Frame size
        '-r', str(int(fps)),  # Frame rate
        '-i', '-',  # Input from pipe
        '-an',  # No audio
        '-vcodec', 'libx264',
        '-pix_fmt', 'yuv420p',
        output_path
    ]

    # Start FFmpeg process
    process = subprocess.Popen(ffmpeg_command, stdin=subprocess.PIPE)

    prev_frame = None
    deviation_timer = 0
    deviation_threshold = 5  # seconds

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Increase contrast of the frame
        contrast_frame = increase_contrast(frame)

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(contrast_frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale frame
        blurred_frame = cv2.GaussianBlur(gray_frame, blur_ksize, blur_sigma)

        if prev_frame is None:
            # For the first frame, there's no previous frame to compare with
            delta_frame = np.zeros_like(blurred_frame)
            thresholded_frame = np.zeros_like(blurred_frame)
        else:
            # Calculate the absolute difference between the current frame and the previous frame
            delta_frame = cv2.absdiff(blurred_frame, prev_frame)

            # Apply threshold to the delta frame
            _, thresholded_frame = cv2.threshold(delta_frame, threshold_value, 255, cv2.THRESH_BINARY)

            # Get the coordinates of the white pixels in the thresholded frame
            white_pixels = np.column_stack(np.where(thresholded_frame == 255))

            # Draw green points at the coordinates of the white pixels
            for pixel in white_pixels:
                frame[pixel[0], pixel[1]] = [0, 255, 0]  # Draw green point

            # Check if smoke flow is within path
            if check_smoke_flow_within_path(white_pixels, path_lines):
                deviation_timer = 0  # Reset deviation timer if smoke flow is within path
                print("Smoke within range")
            else:
                deviation_timer += 1  # Increment deviation timer

            # Check for deviation
            if deviation_timer >= deviation_threshold * fps:  # Convert threshold to frames
                print("Smoke deviation detected!")
                deviation_timer = 0  # Reset timer after detection

        # Draw the defined path lines on the frame
        draw_path_lines(frame, path_lines)

        # Write the frame to FFmpeg process
        process.stdin.write(frame.tobytes())

        # Update previous frame
        prev_frame = blurred_frame

    # Close FFmpeg process
    process.stdin.close()
    process.wait()

    # Release the video capture
    cap.release()
    cv2.destroyAllWindows()

# Function to check if smoke flow is within path
def check_smoke_flow_within_path(smoke_coordinates, path_lines):
    contour_points = np.vstack(path_lines).astype(np.float32)  # Vertically stack all points
    contour = contour_points.reshape((-1, 1, 2))  # Reshape to contour format
    for coord in smoke_coordinates:
        coord_tuple = (float(coord[1]), float(coord[0]))  # Convert to tuple of floats (x, y)
        if cv2.pointPolygonTest(contour, coord_tuple, False) < 0:
            return False
    return True

# Function to draw path lines on a frame
def draw_path_lines(frame, path_lines):
    for line in path_lines:
        if len(line) > 1:
            for i in range(len(line) - 1):
                cv2.line(frame, line[i], line[i + 1], (0, 255, 0), 2)
                cv2.circle(frame, line[i], 5, (255, 0, 0), -1)  # Draw circle at each point
            cv2.circle(frame, line[-1], 5, (255, 0, 0), -1)  # Draw circle at last point

# Load the image
image = cv2.imread(r'C:\Users\25868\Desktop\smoke\Screenshot_20240522-120940.png')

# Open the video file to get the frame size
cap = cv2.VideoCapture('./zoom.mp4')
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# Resize the image to match the video frame size
resized_image = cv2.resize(image, (frame_width, frame_height))

# Create a window and display the resized image
cv2.imshow('Resized Image', resized_image)

# Initialize variables for the path
path_lines = []
first_line_defined = False

# Mouse callback function to capture mouse events
def mouse_callback(event, x, y, flags, param):
    global first_line_defined, path_lines

    if event == cv2.EVENT_LBUTTONDOWN:
        if not first_line_defined:
            if len(path_lines) == 0:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the first line:", (x, y))
        else:
            if len(path_lines) == 1:
                path_lines.append([(x, y)])
            else:
                path_lines[-1].append((x, y))
            print("Point added to the second line:", (x, y))

# Set mouse callback function
cv2.setMouseCallback('Resized Image', mouse_callback)

# Main loop to define the path lines
while True:
    # Read keyboard input
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == 13:  # Enter key
        if not first_line_defined:
            if len(path_lines[-1]) >= 2:
                first_line_defined = True
                print("First line defined:", path_lines[-1])
            else:
                print("Please define at least 2 points for the first line.")
        else:
            if len(path_lines[-1]) >= 2:
                print("Second line defined:", path_lines[-1])
                break
            else:
                print("Please define at least 2 points for the second line.")

    # Draw the defined path lines and points
    if path_lines:
        resized_image_with_lines = resized_image.copy()
        for line in path_lines:
            if len(line) > 1:
                for i in range(len(line) - 1):
                    cv2.line(resized_image_with_lines, line[i], line[i + 1], (0, 255, 0), 2)
                    cv2.circle(resized_image_with_lines, line[i], 5, (255, 0, 0), -1)  # Draw circle at each point
                cv2.circle(resized_image_with_lines, line[-1], 5, (255, 0, 0), -1)  # Draw circle at last point
        cv2.imshow('Resized Image', resized_image_with_lines)

# Close all windows
cv2.destroyAllWindows()

# Path to input video file
input_video_path = './zoom.mp4'

# Path to output video file
output_video_path = './9.mp4'

# Call the function to process the video with a threshold value of 30
process_video(input_video_path, output_video_path, path_lines, threshold_value=10)


Point added to the first line: (206, 47)
Point added to the first line: (206, 156)
Point added to the first line: (229, 310)
Point added to the first line: (270, 444)
Point added to the first line: (339, 580)
Point added to the first line: (408, 679)
First line defined: [(206, 47), (206, 156), (229, 310), (270, 444), (339, 580), (408, 679)]
Point added to the second line: (363, 51)
Point added to the second line: (366, 146)
Point added to the second line: (379, 238)
Point added to the second line: (407, 332)
Point added to the second line: (462, 507)
Point added to the second line: (494, 629)
Point added to the second line: (529, 693)
Second line defined: [(363, 51), (366, 146), (379, 238), (407, 332), (462, 507), (494, 629), (529, 693)]
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke within range
Smoke

In [7]:
import cv2

# Function to adjust the contrast of the frame
def increase_contrast(frame, alpha=2.0, beta=0):
    return cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)

# Function to convert frame to greyscale
def to_greyscale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Function to apply Gaussian blur to the frame
def apply_gaussian_blur(frame, kernel_size=(21, 21), sigma=10):
    return cv2.GaussianBlur(frame, kernel_size, sigma)

# Open the input video
cap = cv2.VideoCapture(r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\output.mp4')

# Get the video's properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\1.mp4', fourcc, fps, (frame_width, frame_height), isColor=False)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Increase contrast
    #frame = increase_contrast(frame)

    # Convert to greyscale
    frame = to_greyscale(frame)

    # Apply Gaussian blur
    frame = apply_gaussian_blur(frame)

    # Write the frame to the output video
    out.write(frame)

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()


In [8]:
import cv2
import numpy as np

def combined_method(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fgbg = cv2.createBackgroundSubtractorMOG2()
    ret, frame1 = cap.read()
    frame1 = cv2.resize(frame1, (800, 500))
    prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    frame_width = 800
    frame_height = 500
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame2 = cap.read()
        if not ret:
            break

        frame2 = cv2.resize(frame2, (800, 500))
        gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Background subtraction
        fgmask = fgbg.apply(frame2)

        # Optical flow
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        mag_thresh = cv2.threshold(magnitude, 2.0, 255, cv2.THRESH_BINARY)[1]

        # Combine the results
        combined_mask = cv2.bitwise_and(fgmask, mag_thresh.astype(np.uint8))

        # Highlight the detected motion
        frame2[combined_mask == 255] = [0, 255, 0]

        # Write the frame to the output video
        out.write(frame2)

        prev_gray = gray

        cv2.imshow('Smoke Movement', frame2)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Call the function with the input video path and the desired output video path
combined_method(r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\output.mp4', r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\1.mp4')


In [9]:
import cv2
import numpy as np

def increase_contrast(frame, clip_limit=2.0, tile_grid_size=(8, 8)):
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    contrast_frame = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return contrast_frame

def combined_method(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fgbg = cv2.createBackgroundSubtractorMOG2()
    ret, frame1 = cap.read()
    frame1 = cv2.resize(frame1, (800, 500))
    prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    frame_width = 800
    frame_height = 500
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame2 = cap.read()
        if not ret:
            break

        frame2 = cv2.resize(frame2, (800, 500))
        gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Increase contrast
        contrast_frame = increase_contrast(frame2)
        gray_contrast = cv2.cvtColor(contrast_frame, cv2.COLOR_BGR2GRAY)

        # Background subtraction
        fgmask = fgbg.apply(contrast_frame)

        # Optical flow
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray_contrast, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        mag_thresh = cv2.threshold(magnitude, 2.0, 255, cv2.THRESH_BINARY)[1]

        # Combine the results
        combined_mask = cv2.bitwise_and(fgmask, mag_thresh.astype(np.uint8))

        # Highlight the detected motion
        frame2[combined_mask == 255] = [0, 255, 0]

        # Write the frame to the output video
        out.write(frame2)

        prev_gray = gray_contrast

        cv2.imshow('Smoke Movement', frame2)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Call the function with the input video path and the desired output video path
combined_method(r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\output.mp4', r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\1.mp4')


In [10]:
import cv2
import numpy as np

def combined_method(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    ret, frame1 = cap.read()
    frame1 = cv2.resize(frame1, (800, 500))
    prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    frame_width = 800
    frame_height = 500
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame2 = cap.read()
        if not ret:
            break

        frame2 = cv2.resize(frame2, (800, 500))
        gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow using Lucas-Kanade method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None,
                                            pyr_scale=0.5, levels=5, winsize=11,
                                            iterations=5, poly_n=5, poly_sigma=1.1, flags=0)
        magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Threshold the magnitude to detect motion
        mag_thresh = cv2.threshold(magnitude, 2.0, 255, cv2.THRESH_BINARY)[1]

        # Highlight the detected motion
        frame2[mag_thresh > 0] = [0, 255, 0]

        # Write the frame to the output video
        out.write(frame2)

        prev_gray = gray

        cv2.imshow('Smoke Movement', frame2)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

combined_method(r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\output.mp4', r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\1.mp4')




In [11]:
import cv2
import numpy as np

def combined_method(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fgbg = cv2.createBackgroundSubtractorMOG2()
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply background subtraction
        fgmask = fgbg.apply(frame)

        # Find contours in the foreground mask
        contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw bounding boxes around detected contours
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Write the frame to the output video
        out.write(frame)

        cv2.imshow('Smoke Movement', frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Call the function with the input video path and the desired output video path
combined_method(r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\output.mp4', r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\1.mp4')



KeyboardInterrupt: 

In [12]:
import cv2
import numpy as np

def combined_method(input_path, output_path):
    cap = cv2.VideoCapture(input_path)

    # Create background subtractor
    fgbg = cv2.createBackgroundSubtractorMOG2(history=500, detectShadows=False)

    # Kernel for morphological operations
    kernel = np.ones((5, 5), np.uint8)

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply background subtraction
        fgmask = fgbg.apply(frame)

        # Perform morphological operations to remove noise
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

        # Find contours in the foreground mask
        contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw bounding boxes around detected contours
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Write the frame to the output video
        out.write(frame)

        cv2.imshow('Smoke Movement', frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Call the function with the input video path and the desired output video path
combined_method(r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\output.mp4', r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\1.mp4')



In [13]:
import cv2
import numpy as np

def edge_detection_method(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (frame_width, frame_height))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply Gaussian Blur
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # Apply Canny Edge Detection
        edges = cv2.Canny(blurred, 50, 150)
        
        # Find contours
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Draw contours on the original frame
        cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)

        # Write the frame to the output video
        out.write(frame)
        cv2.imshow('Edge Detection', frame)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Call the function with the input video path and the desired output video path
edge_detection_method(r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\output.mp4', r'D:\AIML\smoke\smoke_latest\smoke_web\static\videos\1.mp4')
